In [1]:
from entsoe import EntsoePandasClient
import pandas as pd
import datetime

In [12]:
class Price_Agent(): 

    def get_data(self):
        import pandas as pd
        import datetime
        from entsoe import EntsoePandasClient
        start = (pd.Timestamp.today(tz='Europe/Brussels') - datetime.timedelta(days= 1)).normalize()
        end = (start + datetime.timedelta(days = 3)).normalize()
        country_code = 'DE_LU'
        client = EntsoePandasClient(api_key='6f67ccf4-edb3-4100-a850-969c73688627')
        data = client.query_day_ahead_prices(country_code = country_code, start = start, end = end)
        return data


In [16]:
PA = Price_Agent()
data = PA.get_data()

In [17]:
data

2022-12-20 00:00:00+01:00    108.19
2022-12-20 01:00:00+01:00     92.41
2022-12-20 02:00:00+01:00     92.45
2022-12-20 03:00:00+01:00     76.35
2022-12-20 04:00:00+01:00     81.94
                              ...  
2022-12-22 19:00:00+01:00    221.82
2022-12-22 20:00:00+01:00    201.07
2022-12-22 21:00:00+01:00    183.71
2022-12-22 22:00:00+01:00    170.96
2022-12-22 23:00:00+01:00    161.67
Freq: 60T, Length: 72, dtype: float64